In [1]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

In [2]:
path = 'https://raw.githubusercontent.com/FabioHodo/EIB2_ClimateRisk/main/cleaned%20data/data_imputed/imputed_knn/'
df = pd.read_csv(path +'Emissions_GHG_(fromCAIT).csv',index_col='Unnamed: 0' )
df

,Country,Code,1990,1991,1992,1993,1994,1995,1996,1997,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Albania,ALB,11.40,9.04,6.99,6.93,7.77,7.53,7.25,6.54,...,8.54,8.60,8.88,9.33,8.94,9.05,9.54,9.48,9.33,9.90
1,Algeria,DZA,96.00,94.87,94.50,102.03,99.78,103.99,105.30,109.39,...,156.40,166.21,167.32,179.68,189.89,193.59,203.79,211.66,209.41,211.50
2,Angola,AGO,150.61,151.28,151.61,152.39,152.54,156.90,161.39,156.01,...,173.99,180.66,186.83,220.92,185.95,203.34,185.10,207.66,182.19,176.85
3,Antigua and Barbuda,ATG,0.34,0.35,0.43,0.41,0.42,0.44,0.47,0.50,...,1.06,2.02,1.13,1.07,1.29,1.06,1.08,1.10,1.14,1.17
4,Argentina,ARG,323.84,328.83,333.22,335.73,340.81,341.39,344.46,346.24,...,468.33,452.69,440.34,450.10,464.88,462.89,456.74,460.21,469.73,469.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,Vanuatu,VUT,0.47,0.48,0.50,0.53,0.54,0.54,0.57,0.57,...,0.70,0.70,0.71,0.75,0.75,0.75,0.81,0.81,0.84,0.84
163,Venezuela,VEN,303.14,304.92,308.93,315.26,318.02,326.54,331.00,337.59,...,265.49,261.47,291.82,323.30,343.49,373.61,359.46,352.14,352.08,338.14
164,Vietnam,VNM,25.63,28.49,31.00,37.69,43.31,49.46,56.03,63.80,...,209.45,231.71,251.16,235.76,237.24,249.25,263.12,306.26,320.67,319.54
165,Zambia,ZMB,485.43,484.96,484.28,483.00,481.44,478.95,444.46,405.54,...,471.05,412.65,460.49,432.46,455.21,468.32,440.83,473.30,495.17,495.99


In [3]:
def makeMonthly(country, scale = 20):
    '''
    Turn the yearly data into monthly using Brownian bridge algorithm
    Parameter:
        country: a string for the country name
        scale: an integer for the data scaling
    Return:
        the scaled monthly data
    '''
    
    AnnualVal = country*scale
    n = (len(AnnualVal)-1)*12
    monthly = np.zeros(n+1)
    monthlyVal = np.zeros(n)
    for i in range(1, len(AnnualVal)+1):
        monthly[(i-1)*12] = AnnualVal[i-1]

    for i in range(1, len(AnnualVal)): #1-4
        for j in range ((i-1)*12+1, (i)*12): #0 to  47
            monthly[j] = monthly[j-1] + (monthly[(i)*12] - monthly[j-1])*(1/((i*12)-(j-1)))
            + (((i*12-j)*(1)/(i*12-(j-1)))**(1/2))*np.random.randn()
            monthlyVal[j-1] = monthly[j] -monthly[j-1]
    return monthly/scale

In [4]:
def create_dataset(dataset,timestep = 6):
    data_x,data_y = [],[]
    for i in range(len(dataset)-timestep-1):
        a = dataset[i:i+timestep,0]
        data_x.append(a)
        data_y.append(dataset[i+timestep,0])
    return np.array(data_x),np.array(data_y)

In [5]:
def predict(num_prediction, model, test_data, look_back):
    prediction_list = test_data[-look_back:]
    
    for i in range(num_prediction):
        x = prediction_list[-look_back:]
        #print(x)
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back:]
        
    return prediction_list

In [10]:
def FuturePrediction(country,look_back,num_prediction):
    '''
    Make prediction for the data of one country to 2030
    Parameter:
        country: a string that specifies the country name
    Return:
        prediction: a list of values for future prediction of the data for each year
    '''
    data = df[df['Country']==country]
    data = data.loc[:,"1990":"2017"].T
    monthly = makeMonthly(np.array(data), 10)
    monthly = np.reshape(monthly, (-1, 1))
    
    # Feature Scaling
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = scaler.fit_transform(monthly)
    
    #split train and test dataset
    train_size = int(len(data)*0.7)
    test_size = len(data)-train_size
    train_data, test_data = data[0:train_size,:],data[train_size:len(data),:]
    
    x_train, y_train = create_dataset(train_data,timestep = look_back)
    x_test, y_test = create_dataset(test_data,timestep = look_back)
    x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],1)
    x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],1)
    
    # Create the Stacked LSTM model
    model = Sequential()

    model.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 50))
    model.add(Dropout(0.2))
    # Adding the output layer
    model.add(Dense(units = 1))

    # Compiling the RNN
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    # Fitting the RNN to the Training set
    model.fit(x_train, y_train, validation_data = (x_test, y_test) ,epochs = 20, batch_size = 10,verbose =1,
         callbacks=[EarlyStopping(monitor='val_loss', patience=10)])
    
    # Make monthly predictions into the future
    x = predict(num_prediction, model, test_data, look_back)
    prediction = scaler.inverse_transform(x.reshape(-1,1))
    # Make yearly predictions into the future
    annual_prediction = prediction[11:num_prediction:12]
    return annual_prediction

In [11]:
def PredictAll(df,look_back,num_prediction):
    '''
    Make predictions to 2030 for all the countries with valid data
    Parameter:
        df: a dataframe with the data
    Return:
        df: a dataframe with the original data and the predictions
    '''
    countries = df['Country']
    world_prediction = []
    for country in countries:
        #print(FuturePrediction(country,look_back,num_prediction))
        world_prediction.append(FuturePrediction(country,look_back,num_prediction))
    world_prediction = np.squeeze(np.array(world_prediction), axis = (2,))
    years = ['2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025', '2026', '2027', '2028', '2029', '2030']
    #print(world_prediction)
    #print(world_prediction.shape)
    #print(world_prediction[:, 1])
    for i in range(len(years)):
        df[years[i]] = world_prediction[:, i]
    return df


In [12]:
ghg_prediction = PredictAll(df,look_back=5,num_prediction=13*12)
ghg_prediction

Epoch 1/50


KeyboardInterrupt: 

In [ ]:
# Save the dataset as csv for model fitting later
ghg_prediction.to_csv("GHG emission to 2030 (LSTM).csv")